In [ ]:
from selenium import webdriver
from selenium.webdriver import Keys, ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

In [ ]:
exceldata = pd.read_csv("Path to data file.csv")
Youtube_urls = []
Index_datas = []

for idx in exceldata.index:
    youtube = exceldata['Youtube URL'][idx]
    Youtube_urls.append(youtube)
    Index_data = exceldata['Index'][idx]
    Index_datas.append(Index_data)


In [ ]:
driver = webdriver.Chrome()
driver.implicitly_wait(5)

Index = []
Youtube_url = []
Channel_title = []
Channel_sub_count = []
Channel_video_count = []
Channel_view_count=[]
Channel_decription = []
Channel_created_date = []
Number_of_comments_for_featured_video = []
Channel_featured_video_comments = []

driver.maximize_window()

for id,youtube_url in zip(Index_datas,Youtube_urls):
    print(f'@ {id=}, {youtube_url}')
    error_flag = False
    try:
        
        Index.append(id)

        Youtube_url.append(youtube_url)
                                       
        driver.get(youtube_url)           

        tab_list = driver.find_elements(By.ID ,"endpoint")
                
        tab_list[-1].click()
    

        time.sleep(1)
        
        description = driver.find_element(By.XPATH, '//yt-formatted-string[@id="description"]').text
        
        right_column = driver.find_element(By.ID ,"right-column")
        if right_column:
            stats =right_column.find_elements(By.TAG_NAME, "yt-formatted-string")
            created_date = ''.join(stats[1].text.split(" ")[1:])
            view_count = stats[2].text.split(" ")[0]
        else:
            error_flag = True

        channel_title = driver.find_element(By.XPATH, '//yt-formatted-string[contains(@class, "style-scope ytd-channel-name")]').text
        subscriber_count = driver.find_element(By.XPATH, '//yt-formatted-string[@id="subscriber-count"]').text
        number_of_videos = driver.find_element(By.XPATH, '//yt-formatted-string[@id="videos-count"]').text
        
        driver.get(youtube_url)

        try:
            ## Featured Video Section
            driver.find_element(By.XPATH, '//yt-formatted-string[contains(@class, "style-scope ytd-channel-video-player-renderer complex-string")]').click()
            time.sleep(3)

            ActionChains(driver)\
                .send_keys(Keys.PAGE_DOWN)\
                .send_keys(Keys.PAGE_DOWN)\
                .perform()
            
            comment_count_element = driver.find_elements(By.XPATH, '//yt-formatted-string[contains(@class, "count-text style-scope ytd-comments-header-renderer")]')
            comment_count = int(comment_count_element[0].text.split(" ")[0])
            number_comments = comment_count

            iteration = 0
            while comment_count>20 and iteration < 5:
                ActionChains(driver)\
                    .send_keys(Keys.PAGE_DOWN)\
                    .send_keys(Keys.PAGE_DOWN)\
                    .perform()
                time.sleep(2)
                comment_count-=20
                iteration+=1

            comment_elements = driver.find_elements(By.XPATH, '//yt-formatted-string[contains(@id, "content-text")]')
            video_comments = []

            if comment_elements:
                for comment_element in comment_elements[:20]:
                    comment_text = comment_element.text
                    comment_text_santized = comment_text.replace(","," ") # replace commas with space to not break csv
                    video_comments.append(comment_text_santized)

                if len(video_comments) > 0:
                    joined_comments = '|@~~@|'.join(video_comments)

                    Channel_featured_video_comments.append(joined_comments)
                    Number_of_comments_for_featured_video.append(number_comments)
                else:
                    Channel_featured_video_comments.append("")
                    Number_of_comments_for_featured_video.append("0")
            else:
                Channel_featured_video_comments.append("")
                Number_of_comments_for_featured_video.append("0")

        except Exception as e1:
            print("Featured Video Unavailable or comments turned off")
            Channel_featured_video_comments.append("N/A")
            Number_of_comments_for_featured_video.append("N/A")

        
    
        if description:
            Channel_decription.append(description)
        else:
            Channel_decription.append("")

        if channel_title:
            Channel_title.append(channel_title)
        else:
            Channel_title.append("")

        if subscriber_count:
            Channel_sub_count.append(subscriber_count)
        else:
            Channel_sub_count.append("")

        if number_of_videos:
            Channel_video_count.append(number_of_videos)
        else:
            Channel_video_count.append("")
            
        if not error_flag:    
            Channel_created_date.append(created_date)
            Channel_view_count.append(view_count)
        else:
            Channel_created_date.append("")
            Channel_view_count.append("")
            
    except Exception as e:
        print("Exception Occurred")
        Channel_title.append("N/A")
        Channel_sub_count.append("N/A")
        Channel_video_count.append("N/A")
        Channel_decription.append("N/A")
        Channel_created_date.append("N/A")
        Channel_view_count.append("N/A")
        Channel_featured_video_comments.append("N/A")
        Number_of_comments_for_featured_video.append("N/A")


In [ ]:
data = {'Index' : Index,
        'Youtube Url' : Youtube_url,
        'Channel title' : Channel_title,
        'Susbscriber count' : Channel_sub_count,
        'Number of videos posted' : Channel_video_count,
        'Channedl created date' : Channel_created_date,
        'Overalll view count for the channel' : Channel_view_count,
        'Channel description' : Channel_decription,       
        'Number of comments for featured video' : Number_of_comments_for_featured_video,
        'Featured Video Comment': Channel_featured_video_comments }

yt_df = pd.DataFrame(data)
yt_df

In [ ]:
yt_df.to_csv('Youtube_data.csv')